<a href="https://colab.research.google.com/github/fejes99/PPPO-projekat/blob/main/PPPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Univerzitet u Novom Sadu**

**Fakultet tehničkih nauka**

**Departman za industrijsko inženjerstvo i menadžment**

**Inženjerstvo informacionih sistema**

**Predmet**: Principi prezentacije i prepoznavanja oblika

**Tema**: Segmentacija

**Student**: David Feješ IT-70/2017


# UVOD

Segmentacija predstavlja ključni aspekt analize slika. Ovaj rad se bavi segmentacijom podataka primenjujući UNet arhitekturu na skupu podataka koji se bavi detekcijom soli u seizmičkim istraživanjima. Segmentacija je tehnika izdvajanja i analize regiona od interesa na slikama.


# SEGMENTACIJA

U ovom radu primenjena je UNet arhitektura za segmentaciju slika. UNet je duboka neuronska mreža i ima sposobnost preciznog izdvajanja regiona. Ova arhitektura se sastoji iz konvolucionih slojeva za ekstrakciju karakteristika koji služe za izdvajanje karakteristika slika, kao i up-scaling slojeva koji omogućavaju generisanje segmentacionih mapa.

# Metrike za evaluaciju

Za evaluaciju razultata koriste se različite tehnike. Precision, recall i F1-Score su klasične metrike koje mere tačnost segmentacije u odnosu na tačno izdvojene regione sa solju. Precision se odnosi na broj pravilno identifikovanih piksela sa solju u odnosu na ukupan broj identifikovanih piksela. Recall meri koliko tačnih piksela sa solju je prepoznato u odnosu na ukupan broj piksela sa solju. F1-Score kombinuje precision i recall kako bi dao celokupnu ocenu tačnosti segmentacije.

# Zaključak

 Segmentacija podataka ima široku primenu u mnogim oblastima, a ovaj rad se specifično fokusira na problem detekcije soli na slikama iz seizmičkih istraživanja.

Ovo istraživanje omogućava bolje razumevanje tehnikа segmentacije slika i primene dubokih neuronskih mreža u analizi seizmičkih podataka. Takođe, naglašava važnost precizne segmentacije u detekciji soli, što može biti od suštinskog značaja u seizmičkim istraživanjima i drugim oblastima gde je detekcija regiona od interesa od velikog značaja.


In [ ]:
!pip install tensorflow-gpu

Importovanje neophodnih biblioteka koje će biti potrebne za dalju izradu projekta.

In [ ]:
import os
import sys
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, MaxPooling2D, GlobalMaxPool2D, Conv2D, Conv2DTranspose, concatenate, add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix


Povezivanje Google Drive-a.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Metoda load_image koristi se za učitavanje i obradu fotografije. Ulazni parametar je putanja slike. Učitana slika se konvertuje u crno-belu fotografiju i biće promenjena veličina slike na 128x128 piksela.

In [ ]:
def load_image(img_path, show=False):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (128,128))
    img_tensor = img_to_array(img)

    return img_tensor

Učitavaju se slike i maske, normalizuju i skladište u liste.

In [ ]:
x = []
y = []

def get_image(path):
    data =[]
    for subdir, dirs, files in os.walk(path):
        for f in files:
            path = os.path.join(subdir, f)
            img = load_image(path)
            img = img/255.0
            data.append(img)
    return data


x = get_image(r'/content/gdrive/My Drive/data/train/images')
y = get_image(r'/content/gdrive/My Drive/data/train/masks')

x = np.asarray(x)
y = np.asarray(y)

U ovom bloku se vrši podela slika i maski na podatke za obuku i podatke za validaciju u odnosu 80% za obuku i 20% za validaciju.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
print(x.shape)
print(y.shape)

Vizuelizacija nasumično odabrane slike i njene maske.

In [ ]:
ix = random.randint(0, len(X_train))
has_mask = y_train[ix].max() > 0

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 5))

ax1.imshow(X_train[ix, ..., 0], cmap = 'seismic', interpolation = 'bilinear')
if has_mask:
    ax1.contour(y_train[ix].squeeze(), colors = 'k', linewidths = 5, levels = [0.5])
ax1.set_title('Seismic')

ax2.imshow(y_train[ix].squeeze(), cmap = 'gray', interpolation = 'bilinear')
ax2.set_title('Salt')

In [ ]:
def UNet(input_img):
    # Prvi sloj
    c1 = Conv2D(32, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(input_img)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)

    c1 = Conv2D(32, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(c1)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)

    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(0.1)(p1)

    # Drugi sloj
    c2 = Conv2D(64, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)

    c2 = Conv2D(64, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(c2)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)

    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(0.1)(p2)

    # Treći sloj
    c3 = Conv2D(128, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)

    c3 = Conv2D(128, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(c3)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)

    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(0.2)(p3)

    # Četvrti sloj
    c4 = Conv2D(256, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)

    c4 = Conv2D(256, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(c4)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)

    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
    p4 = Dropout(0.2)(p4)

    # Srednji sloj
    c5 = Conv2D(512, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(p4)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)

    c5 = Conv2D(512, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(c5)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)

    # Transponovani konvolucioni slojevi za dekodiranje
    u6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(0.2)(u6)

    c6 = Conv2D(256, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(u6)
    c6 = BatchNormalization()(c6)
    c6 = Activation('relu')(c6)

    c6 = Conv2D(256, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(c6)
    c6 = BatchNormalization()(c6)
    c6 = Activation('relu')(c6)

    u7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(0.2)(u7)

    c7 = Conv2D(128, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(u7)
    c7 = BatchNormalization()(c7)
    c7 = Activation('relu')(c7)

    c7 = Conv2D(128, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(c7)
    c7 = BatchNormalization()(c7)
    c7 = Activation('relu')(c7)

    u8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(0.1)(u8)

    c8 = Conv2D(64, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(u8)
    c8 = BatchNormalization()(c8)
    c8 = Activation('relu')(c8)

    c8 = Conv2D(64, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(c8)
    c8 = BatchNormalization()(c8)
    c8 = Activation('relu')(c8)

    u9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(0.1)(u9)

    c9 = Conv2D(32, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(u9)
    c9 = BatchNormalization()(c9)
    c9 = Activation('relu')(c9)

    c9 = Conv2D(32, kernel_size=(3, 3), kernel_initializer='he_normal', padding='same')(c9)
    c9 = BatchNormalization()(c9)
    c9 = Activation('relu')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(input_img, outputs)
    return model

Kompajliranje UNet modela sa Adam optimizatorom i binarnim unakrsnim entropijskim gubitkom. Nakon toga model se sumira i može se koristiti za segmentaciju slika.

In [ ]:
input_img = Input((128, 128, 1), name='img')
model = UNet(input_img)
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["acc"])
model.summary()

Definisanje Callback funkcija:


1.   EarlyStopping - prati performanse modela tokom treninga i prekida trening ako se gubitak na validacionom skupu ne poboljšava u narednih 10 epoha. Pomaže sprečavanju overfitting-a i trening se završava kada model prestane da se poboljšava.
2.   ReduceLROnPlateau - smanjuje stopu učenja ako se rezultat na validacionom skupu ne poboljša u narednih 5 epoha.
3.   ModelCheckpoint - čuva najbolji model



In [ ]:
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint('/content/gdrive/My Drive/best_model_during_training.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

Trening samog modela.

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=50, callbacks=callbacks,\
                    validation_data=(X_valid, y_valid))

In [ ]:
model.save('/content/gdrive/My Drive/final_model_after_training.h5')

U ovom bloku model vrši predikciju na skupu za obuku i skupu za validaciju.

In [ ]:
preds_train = model.predict(X_train, verbose=1)
preds_val = model.predict(X_valid, verbose=1)

In [ ]:
# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

def calculate_metrics(y_true, y_pred):
    # Threshold the ground truth masks to binary format
    y_true_bin = (y_true > 0.5).astype(np.uint8)

    # Threshold the predictions to binary format
    y_pred_bin = (y_pred > 0.5).astype(np.uint8)

    # Calculate confusion matrix
    confusion = confusion_matrix(y_true_bin.ravel(), y_pred_bin.ravel())

    # Calculate precision, recall, and F1-Score
    precision = precision_score(y_true_bin.ravel(), y_pred_bin.ravel())
    recall = recall_score(y_true_bin.ravel(), y_pred_bin.ravel())
    f1 = f1_score(y_true_bin.ravel(), y_pred_bin.ravel())

    # Convert precision to percentage
    precision_percent = precision * 100

    return confusion, precision, recall, f1, precision_percent

# Calculate metrics for training set
confusion_train, precision_train, recall_train, f1_train, precision_train_percent = calculate_metrics(y_train, preds_train)

# Calculate metrics for validation set
confusion_val, precision_val, recall_val, f1_val, precision_val_percent = calculate_metrics(y_valid, preds_val)

# Print and visualize the metrics
print("Training Precision:", precision_train)
print("Training Precision Percentage:", precision_train_percent, "%")
print("Training Recall:", recall_train)
print("Training F1-Score:", f1_train)

print("Validation Precision:", precision_val)
print("Validation Precision Percentage:", precision_val_percent, "%")
print("Validation Recall:", recall_val)
print("Validation F1-Score:", f1_val)

# Visualize the confusion matrix
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].matshow(confusion_train, cmap=plt.cm.Blues, alpha=0.7)
axes[0].set_title("Training Confusion Matrix")
axes[1].matshow(confusion_val, cmap=plt.cm.Blues, alpha=0.7)
axes[1].set_title("Validation Confusion Matrix")
plt.show()


In [ ]:
def plot_sample(X, y, preds, binary_preds):
    ix = random.randint(0, len(X))

    has_mask = y[ix].max() > 0

    fig, ax = plt.subplots(1, 4, figsize=(20, 10))
    ax[0].imshow(X[ix, ..., 0], cmap='seismic')
    if has_mask:
        ax[0].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[0].set_title('Seismic')

    ax[1].imshow(y[ix].squeeze())
    ax[1].set_title('Salt')

    ax[2].imshow(preds[ix].squeeze(), vmin=0, vmax=1)
    if has_mask:
        ax[2].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[2].set_title('Salt Predicted')

    ax[3].imshow(binary_preds[ix].squeeze(), vmin=0, vmax=1)
    if has_mask:
        ax[3].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[3].set_title('Salt Predicted binary');

In [ ]:
plot_sample(X_train, y_train, preds_train, preds_train_t)